In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement intel-extension-for-pytorch (from versions: none)
ERROR: No matching distribution found for intel-extension-for-pytorch


3

In [18]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
from fnapproxto import fnapproxto
import pandas as pd
import numpy as np

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# import csv data with pd
data = pd.read_csv('generated_dataset.csv')
mean = np.array(data).mean(axis=0, keepdims=True)
std = np.array(data).std(axis=0, keepdims=True)
xvarnum = 6
for i in range(xvarnum):
    data.iloc[:, i] = (data.iloc[:, i] - mean[0][i]) / std[0][i]


Using cpu device
<bound method NDFrame.head of              x1        x2        x3        x4        x5        x6  \
0      0.249496  1.651955 -0.518763  0.186928  0.347909 -0.286217   
1      0.164447  0.130632  0.729240 -0.011466 -0.333234 -0.997897   
2     -0.494170  1.240444  1.433147  0.151573  0.001301 -0.645137   
3      1.693532  0.500309 -0.518393 -1.146228 -0.304018  0.492199   
4      0.640281 -1.470265 -0.521338  1.002532 -1.641815 -1.618025   
...         ...       ...       ...       ...       ...       ...   
99995 -0.569228  0.464088 -0.177436 -1.567349  0.186987 -1.317814   
99996 -0.768586  1.473541 -1.370753  0.167335 -1.615549  0.419048   
99997  0.275200  1.346814  0.277645 -1.627968 -1.512860  0.507975   
99998  0.506362  0.151971 -1.478689  1.199874  1.702765 -0.689720   
99999  0.513383  1.655695 -0.716390 -1.013162  1.262534  0.992421   

                y1         y2  
0       404.006021  26.117891  
1       234.866294  41.490010  
2       363.410767  56.83642

In [75]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
from fnapproxto import fnapproxto
import pandas as pd
import numpy as np

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# import csv data with pd
data = pd.read_csv('generated_dataset.csv')
mean = np.array(data).mean(axis=0, keepdims=True)
std = np.array(data).std(axis=0, keepdims=True)
xvarnum = 6
for i in range(xvarnum):
    data.iloc[:, i] = (data.iloc[:, i] - mean[0][i]) / std[0][i]

print(data.shape)
# show first six rows of data
#print(data.head())

train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)

# batch
batch_num = 10

# separate the data into batches then to tensors 
def create_batches(data, num_batches):
    data = np.array(data)
    np.random.shuffle(data)
    batch_size = len(data) // num_batches
    batches = [data[i:i + batch_size] for i in range(0, len(data), batch_size)]
    return [(torch.tensor(batch[:,0:6], dtype=torch.float32),torch.tensor(batch[:,-2:], dtype=torch.float32)) for batch in batches]


batched_training_data = create_batches(train_data, batch_num)
batched_test_data = create_batches(test_data, 1)



# initiate the model
state_size = 6
action_size = 2
hidden_size = 31
learning_rate = 0.0001
fnapprox = fnapproxto(state_size=state_size, action_size=action_size, hidden_size=hidden_size, learning_rate=learning_rate).to(device)


# run the training loops
epochs = 1
testloss = []
for t in range(epochs):
    fnapprox.train_model(batched_training_data,device)
    if t % 100 == 0:
        print(f"Epoch {t+1}")
        loss = fnapprox.test_model(batched_test_data,device)
        testloss.append(loss)
        print(f"Test loss: {loss}\r")
print("Done!")


Using cpu device
(100000, 8)
Epoch 1
Test loss: 13659144.0
Done!


In [3]:
# additional training loops
epochs = 100000
for t in range(epochs):
    fnapprox.train_model(batched_training_data,device)
    if t % 100 == 0:
        print(f"Epoch {t+1}")
        loss = fnapprox.test_model(batched_test_data,device)
        testloss.append(loss)
        print(f"Test loss: {loss}\r")
print("Done!")


Epoch 71401
Test loss: 3.9733469486236572
Epoch 71501
Test loss: 3.9729602336883545
Epoch 71601
Test loss: 3.972149133682251
Epoch 71701
Test loss: 3.9713077545166016
Epoch 71801
Test loss: 3.97056245803833
Epoch 71901
Test loss: 3.9699628353118896
Epoch 72001
Test loss: 3.9693171977996826
Epoch 72101
Test loss: 3.9687840938568115
Epoch 72201
Test loss: 3.968243360519409
Epoch 72301
Test loss: 3.9677555561065674
Epoch 72401
Test loss: 3.966158628463745
Epoch 72501
Test loss: 3.966482400894165
Epoch 72601
Test loss: 3.965475082397461
Epoch 72701
Test loss: 3.965581178665161
Epoch 72801
Test loss: 3.9645586013793945
Epoch 72901
Test loss: 3.964478015899658
Epoch 73001
Test loss: 3.9638988971710205
Epoch 73101
Test loss: 3.9626383781433105
Epoch 73201
Test loss: 3.962134838104248
Epoch 73301
Test loss: 3.9623007774353027
Epoch 73401
Test loss: 3.961516857147217
Epoch 73501
Test loss: 3.9607796669006348
Epoch 73601
Test loss: 3.960557460784912
Epoch 73701
Test loss: 3.9594297409057617
Epoc

In [76]:
torch.save(fnapprox.state_dict(), "fnapprox-untrained.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [84]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
from fnapproxto import fnapproxto
import pandas as pd
import numpy as np

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# load the trained model
state_size = 6
action_size = 2
hidden_size = 31
learning_rate = 0.0001
trained = fnapproxto(state_size=state_size, action_size=action_size, hidden_size=hidden_size, learning_rate=learning_rate).to(device)
trained.load_state_dict(torch.load("fnapprox.pth", weights_only=True))

# load a untrained model.
untrained = fnapproxto(state_size=state_size, action_size=action_size, hidden_size=hidden_size, learning_rate=learning_rate).to(device)
untrained.load_state_dict(torch.load("fnapprox-untrained.pth", weights_only=True))


# import csv data with pd
data = pd.read_csv('generated_dataset_test.csv')
mean = np.array(data).mean(axis=0, keepdims=True)
std = np.array(data).std(axis=0, keepdims=True)
xvarnum = 6
for i in range(xvarnum):
    data.iloc[:, i] = (data.iloc[:, i] - mean[0][i]) / std[0][i]

# batch
batch_num = 1

# separate the data into batches then to tensors 
def create_batches(data, num_batches):
    data = np.array(data)
    np.random.shuffle(data)
    batch_size = len(data) // num_batches
    batches = [data[i:i + batch_size] for i in range(0, len(data), batch_size)]
    return [(torch.tensor(batch[:,0:6], dtype=torch.float32),torch.tensor(batch[:,-2:], dtype=torch.float32)) for batch in batches]


training_data = create_batches(data, 1)









Using cpu device


In [ ]:
# predict using trained and untrained models
trainedpred = trained(training_data[0][0])
untrainedpred = untrained(training_data[0][0])

# get average difference from the actual values
traineddiff = torch.mean(torch.abs(trainedpred - training_data[0][1]))
untraineddiff = torch.mean(torch.abs(untrainedpred - training_data[0][1]))
traineddiffvar = torch.var(torch.abs(trainedpred - training_data[0][1]))
untraineddiffvar = torch.var(torch.abs(untrainedpred - training_data[0][1]))

print(traineddiff.item())
print(traineddiffvar.item())
print(untraineddiff.item())
print(untraineddiffvar.item())

4.616389751434326
50.58995819091797
1234.2237548828125
11786070.0


In [111]:
import copy
trained_copy = copy.deepcopy(trained)
trainedpred = trained_copy(training_data[0][0])
traineddiff = torch.mean(torch.abs(trainedpred - training_data[0][1]))
untraineddiff = torch.mean(torch.abs(untrainedpred - training_data[0][1]))

print(traineddiff.item())


4.616389751434326
